# cov_vaccine_degradation

In [5]:
import os
import argparse
import torch

from data.dataloaders import get_multimodal_loaders
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.multimodel import build_model
from trainer import RegressionTrainer


def _has_any_embeddings(emb_dir) -> bool:
    if not os.path.isdir(emb_dir):
        return False
    try:
        for fn in os.listdir(emb_dir):
            if fn.endswith(".pt"):
                return True
    except FileNotFoundError:
        return False
    return False


def main(name: str, dataset: str, max_len: int, batch_size: int, epochs: int):
    config = load_config(f"{name}.yml")
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"

    dna_dir = f"embeddings/{config['Dataset']}/DNA/maxlen{max_len}"
    rna_dir = f"embeddings/{config['Dataset']}/RNA/maxlen{max_len}"
    prot_dir = f"embeddings/{config['Dataset']}/Protein/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_dna = not _has_any_embeddings(dna_dir)
    need_rna = not _has_any_embeddings(rna_dir)
    need_prot = not _has_any_embeddings(prot_dir)
    need_embeddings = need_dna or need_rna or need_prot

    if need_filtered_csv or need_embeddings:
        print("Embeddings and/or filtered CSV not found, calculating...")
        for modality in ("DNA", "RNA", "Protein"):
            calculate_embeddings(
                dataset=config["Dataset"],
                modality=modality,
                device=config["device"],
                max_len=max_len,
            )

    print("=" * 60)
    print("Training Configuration (Multimodal):")
    print(f"  Config name: {config.get('name', name)}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Max Len (filter): {max_len}")
    print(f"  Fusion: {config.get('fusion_type', 'concat')}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print(f"  Device: {config['device']}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_multimodal_loaders(
        config["Dataset"],
        batch_size=batch_size,
        max_len=max_len,
    )

    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = RegressionTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config.get('name', name)}/{config['Dataset']}",
    )

    # entropy reg for MIL (nested config with backwards-compatible fallback)
    lam_entropy = None
    if isinstance(config.get("trainer", None), dict):
        lam_entropy = config["trainer"].get("lam_entropy", None)
    if lam_entropy is None:
        lam_entropy = config.get("lam_entropy", None)

    if lam_entropy is not None:
        trainer.lam_entropy = float(lam_entropy)
        if trainer.lam_entropy > 0:
            print(f"Using MIL entropy regularization: lam_entropy={trainer.lam_entropy}")

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=epochs)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train multimodal model (DNA+RNA+Protein)")
    parser.add_argument(
        "--name",
        type=str,
        default="fusion_concat",
        help="Config file name (without .yml). Options: fusion_concat, fusion_mil, fusion_xattn",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="cov_vaccine_degradation",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1000,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=32,
        help="Batch size.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=500,
        help="Number of training epochs.",
    )
    args, _ = parser.parse_known_args()

    main(
        name=args.name,
        dataset=args.dataset,
        max_len=args.max_len,
        batch_size=args.batch_size,
        epochs=args.epochs,
    )

Training Configuration (Multimodal):
  Config name: fusion_concat
  Dataset: cov_vaccine_degradation
  Max Len (filter): 1000
  Fusion: concat
  Batch size: 32
  Epochs: 500
  Device: cpu

Loading data...

Initializing model...
Total number of parameters: 2497753
Trainable parameters: 2497753
Non-trainable parameters: 0

Initializing trainer...

Starting training...

Starting training for 500 epochs...
Device: cpu
Save directory: plots/fusion_concat/cov_vaccine_degradation


Validating: 100%|███████████████████████████████| 13/13 [00:01<00:00, 11.62it/s]


Epoch 1/500
  Train Loss: 1.1657, MSE: 1.1657, Spearman: 0.2603, LR: 0.000030
  Val Loss: 0.4941, MSE: 0.4941, Spearman: 0.6864


Validating: 100%|███████████████████████████████| 13/13 [00:01<00:00, 11.31it/s]


Epoch 2/500
  Train Loss: 1.0804, MSE: 1.0804, Spearman: 0.3997, LR: 0.000030
  Val Loss: 0.4164, MSE: 0.4164, Spearman: 0.7122


...

Epoch 47/500
  Train Loss: 0.2677, MSE: 0.2677, Spearman: 0.8838, LR: 0.000030
  Val Loss: 0.2325, MSE: 0.2325, Spearman: 0.8046


Validating: 100%|███████████████████████████████| 13/13 [00:00<00:00, 20.35it/s]


Epoch 48/500
  Train Loss: 0.2589, MSE: 0.2589, Spearman: 0.8972, LR: 0.000030
  Val Loss: 0.2472, MSE: 0.2472, Spearman: 0.8022

Early stopping triggered after 48 epochs!
Best validation loss: 0.1942

Training completed! Best validation loss: 0.1942


Validating: 100%|███████████████████████████████| 13/13 [00:01<00:00, 12.03it/s]



Test Results:
  Loss: 0.2283
  MSE: 0.2283
  Spearman: 0.7933

Training completed!


In [3]:
import os
import argparse
import torch

from data.dataloaders import get_multimodal_loaders
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.multimodel import build_model
from trainer import RegressionTrainer


def _has_any_embeddings(emb_dir) -> bool:
    if not os.path.isdir(emb_dir):
        return False
    try:
        for fn in os.listdir(emb_dir):
            if fn.endswith(".pt"):
                return True
    except FileNotFoundError:
        return False
    return False


def main(name: str, dataset: str, max_len: int, batch_size: int, epochs: int):
    config = load_config(f"{name}.yml")
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"

    dna_dir = f"embeddings/{config['Dataset']}/DNA/maxlen{max_len}"
    rna_dir = f"embeddings/{config['Dataset']}/RNA/maxlen{max_len}"
    prot_dir = f"embeddings/{config['Dataset']}/Protein/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_dna = not _has_any_embeddings(dna_dir)
    need_rna = not _has_any_embeddings(rna_dir)
    need_prot = not _has_any_embeddings(prot_dir)
    need_embeddings = need_dna or need_rna or need_prot

    if need_filtered_csv or need_embeddings:
        print("Embeddings and/or filtered CSV not found, calculating...")
        for modality in ("DNA", "RNA", "Protein"):
            calculate_embeddings(
                dataset=config["Dataset"],
                modality=modality,
                device=config["device"],
                max_len=max_len,
            )

    print("=" * 60)
    print("Training Configuration (Multimodal):")
    print(f"  Config name: {config.get('name', name)}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Max Len (filter): {max_len}")
    print(f"  Fusion: {config.get('fusion_type', 'concat')}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print(f"  Device: {config['device']}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_multimodal_loaders(
        config["Dataset"],
        batch_size=batch_size,
        max_len=max_len,
    )

    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = RegressionTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config.get('name', name)}/{config['Dataset']}",
    )

    # entropy reg for MIL (nested config with backwards-compatible fallback)
    lam_entropy = None
    if isinstance(config.get("trainer", None), dict):
        lam_entropy = config["trainer"].get("lam_entropy", None)
    if lam_entropy is None:
        lam_entropy = config.get("lam_entropy", None)

    if lam_entropy is not None:
        trainer.lam_entropy = float(lam_entropy)
        if trainer.lam_entropy > 0:
            print(f"Using MIL entropy regularization: lam_entropy={trainer.lam_entropy}")

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=epochs)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train multimodal model (DNA+RNA+Protein)")
    parser.add_argument(
        "--name",
        type=str,
        default="fusion_mil",
        help="Config file name (without .yml). Options: fusion_concat, fusion_mil, fusion_xattn",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="cov_vaccine_degradation",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1000,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=32,
        help="Batch size.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=500,
        help="Number of training epochs.",
    )
    args, _ = parser.parse_known_args()

    main(
        name=args.name,
        dataset=args.dataset,
        max_len=args.max_len,
        batch_size=args.batch_size,
        epochs=args.epochs,
    )

Training Configuration (Multimodal):
  Config name: fusion_mil
  Dataset: cov_vaccine_degradation
  Max Len (filter): 1000
  Fusion: mil
  Batch size: 32
  Epochs: 500
  Device: cpu

Loading data...

Initializing model...
Total number of parameters: 3185374
Trainable parameters: 3185374
Non-trainable parameters: 0

Initializing trainer...
Using MIL entropy regularization: lam_entropy=0.01

Starting training...

Starting training for 500 epochs...
Device: cpu
Save directory: plots/fusion_mil/cov_vaccine_degradation


Validating: 100%|███████████████████████████████| 13/13 [00:01<00:00, 10.86it/s]


Epoch 1/500
  Train Loss: 1.2053, MSE: 1.1944, Spearman: 0.1818, LR: 0.000030
  Val Loss: 0.5678, MSE: 0.5569, Spearman: 0.6082


Validating: 100%|███████████████████████████████| 13/13 [00:00<00:00, 13.63it/s]


Epoch 2/500
  Train Loss: 1.1636, MSE: 1.1528, Spearman: 0.2765, LR: 0.000030
  Val Loss: 0.4996, MSE: 0.4890, Spearman: 0.6371


...

Epoch 32/500
  Train Loss: 0.3168, MSE: 0.3142, Spearman: 0.8548, LR: 0.000030
  Val Loss: 0.3827, MSE: 0.3802, Spearman: 0.6590


Validating: 100%|███████████████████████████████| 13/13 [00:00<00:00, 13.74it/s]


Epoch 33/500
  Train Loss: 0.3075, MSE: 0.3051, Spearman: 0.8625, LR: 0.000030
  Val Loss: 0.3672, MSE: 0.3647, Spearman: 0.6751

Early stopping triggered after 33 epochs!
Best validation loss: 0.2619

Training completed! Best validation loss: 0.2619


Validating: 100%|███████████████████████████████| 13/13 [00:01<00:00, 11.38it/s]



Test Results:
  Loss: 0.3121
  MSE: 0.3055
  Spearman: 0.7007

Training completed!


In [4]:
import os
import argparse
import torch

from data.dataloaders import get_multimodal_loaders
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.multimodel import build_model
from trainer import RegressionTrainer


def _has_any_embeddings(emb_dir) -> bool:
    if not os.path.isdir(emb_dir):
        return False
    try:
        for fn in os.listdir(emb_dir):
            if fn.endswith(".pt"):
                return True
    except FileNotFoundError:
        return False
    return False


def main(name: str, dataset: str, max_len: int, batch_size: int, epochs: int):
    config = load_config(f"{name}.yml")
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"

    dna_dir = f"embeddings/{config['Dataset']}/DNA/maxlen{max_len}"
    rna_dir = f"embeddings/{config['Dataset']}/RNA/maxlen{max_len}"
    prot_dir = f"embeddings/{config['Dataset']}/Protein/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_dna = not _has_any_embeddings(dna_dir)
    need_rna = not _has_any_embeddings(rna_dir)
    need_prot = not _has_any_embeddings(prot_dir)
    need_embeddings = need_dna or need_rna or need_prot

    if need_filtered_csv or need_embeddings:
        print("Embeddings and/or filtered CSV not found, calculating...")
        for modality in ("DNA", "RNA", "Protein"):
            calculate_embeddings(
                dataset=config["Dataset"],
                modality=modality,
                device=config["device"],
                max_len=max_len,
            )

    print("=" * 60)
    print("Training Configuration (Multimodal):")
    print(f"  Config name: {config.get('name', name)}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Max Len (filter): {max_len}")
    print(f"  Fusion: {config.get('fusion_type', 'concat')}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print(f"  Device: {config['device']}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_multimodal_loaders(
        config["Dataset"],
        batch_size=batch_size,
        max_len=max_len,
    )

    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = RegressionTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config.get('name', name)}/{config['Dataset']}",
    )

    # entropy reg for MIL (nested config with backwards-compatible fallback)
    lam_entropy = None
    if isinstance(config.get("trainer", None), dict):
        lam_entropy = config["trainer"].get("lam_entropy", None)
    if lam_entropy is None:
        lam_entropy = config.get("lam_entropy", None)

    if lam_entropy is not None:
        trainer.lam_entropy = float(lam_entropy)
        if trainer.lam_entropy > 0:
            print(f"Using MIL entropy regularization: lam_entropy={trainer.lam_entropy}")

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=epochs)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train multimodal model (DNA+RNA+Protein)")
    parser.add_argument(
        "--name",
        type=str,
        default="fusion_xattn",
        help="Config file name (without .yml). Options: fusion_concat, fusion_mil, fusion_xattn",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="cov_vaccine_degradation",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1000,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=32,
        help="Batch size.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=500,
        help="Number of training epochs.",
    )
    args, _ = parser.parse_known_args()

    main(
        name=args.name,
        dataset=args.dataset,
        max_len=args.max_len,
        batch_size=args.batch_size,
        epochs=args.epochs,
    )

Training Configuration (Multimodal):
  Config name: fusion_xattn
  Dataset: cov_vaccine_degradation
  Max Len (filter): 1000
  Fusion: xattn
  Batch size: 32
  Epochs: 500
  Device: cpu

Loading data...

Initializing model...
Total number of parameters: 9315473
Trainable parameters: 9315473
Non-trainable parameters: 0

Initializing trainer...

Starting training...

Starting training for 500 epochs...
Device: cpu
Save directory: plots/fusion_xattn/cov_vaccine_degradation


Validating: 100%|███████████████████████████████| 13/13 [00:02<00:00,  4.87it/s]


Epoch 1/500
  Train Loss: 1.2714, MSE: 1.2714, Spearman: 0.0256, LR: 0.000030
  Val Loss: 0.5419, MSE: 0.5419, Spearman: 0.5023


Validating: 100%|███████████████████████████████| 13/13 [00:02<00:00,  4.86it/s]


Epoch 2/500
  Train Loss: 1.1629, MSE: 1.1629, Spearman: 0.2077, LR: 0.000030
  Val Loss: 0.4848, MSE: 0.4848, Spearman: 0.5456


...

Epoch 37/500
  Train Loss: 0.5716, MSE: 0.5716, Spearman: 0.6922, LR: 0.000030
  Val Loss: 0.2879, MSE: 0.2879, Spearman: 0.7518


Validating: 100%|███████████████████████████████| 13/13 [00:02<00:00,  4.99it/s]


Epoch 38/500
  Train Loss: 0.4182, MSE: 0.4182, Spearman: 0.7463, LR: 0.000030
  Val Loss: 0.3069, MSE: 0.3069, Spearman: 0.7615

Early stopping triggered after 38 epochs!
Best validation loss: 0.2785

Training completed! Best validation loss: 0.2785


Validating: 100%|███████████████████████████████| 13/13 [00:02<00:00,  5.05it/s]



Test Results:
  Loss: 0.3011
  MSE: 0.3011
  Spearman: 0.7285

Training completed!


---

# ecoli_proteins

In [ ]:
import os
import argparse
import torch

from data.dataloaders import get_multimodal_loaders
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.multimodel import build_model
from trainer import RegressionTrainer


def _has_any_embeddings(emb_dir) -> bool:
    if not os.path.isdir(emb_dir):
        return False
    try:
        for fn in os.listdir(emb_dir):
            if fn.endswith(".pt"):
                return True
    except FileNotFoundError:
        return False
    return False

def main(name: str, dataset: str, max_len: int, batch_size: int, epochs: int):
    config = load_config(f"{name}.yml")
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"

    dna_dir = f"embeddings/{config['Dataset']}/DNA/maxlen{max_len}"
    rna_dir = f"embeddings/{config['Dataset']}/RNA/maxlen{max_len}"
    prot_dir = f"embeddings/{config['Dataset']}/Protein/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_dna = not _has_any_embeddings(dna_dir)
    need_rna = not _has_any_embeddings(rna_dir)
    need_prot = not _has_any_embeddings(prot_dir)
    need_embeddings = need_dna or need_rna or need_prot

    if need_filtered_csv or need_embeddings:
        print("Embeddings and/or filtered CSV not found, calculating...")
        for modality in ("DNA", "RNA", "Protein"):
            calculate_embeddings(
                dataset=config["Dataset"],
                modality=modality,
                device=config["device"],
                max_len=max_len,
            )

    print("=" * 60)
    print("Training Configuration (Multimodal):")
    print(f"  Config name: {config.get('name', name)}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Max Len (filter): {max_len}")
    print(f"  Fusion: {config.get('fusion_type', 'concat')}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print(f"  Device: {config['device']}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_multimodal_loaders(
        config["Dataset"],
        batch_size=batch_size,
        max_len=max_len,
    )

    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = ClassificationTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config.get('name', name)}/{config['Dataset']}",
    )

    # entropy reg for MIL (nested config with backwards-compatible fallback)
    lam_entropy = None
    if isinstance(config.get("trainer", None), dict):
        lam_entropy = config["trainer"].get("lam_entropy", None)
    if lam_entropy is None:
        lam_entropy = config.get("lam_entropy", None)

    if lam_entropy is not None:
        trainer.lam_entropy = float(lam_entropy)
        if trainer.lam_entropy > 0:
            print(f"Using MIL entropy regularization: lam_entropy={trainer.lam_entropy}")

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=epochs)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train multimodal model (DNA+RNA+Protein)")
    parser.add_argument(
        "--name",
        type=str,
        default="fusion_concat",
        help="Config file name (without .yml). Options: fusion_concat, fusion_mil, fusion_xattn",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="ecoli_proteins",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1000,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=32,
        help="Batch size.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=500,
        help="Number of training epochs.",
    )
    args, _ = parser.parse_known_args()

    main(
        name=args.name,
        dataset=args.dataset,
        max_len=args.max_len,
        batch_size=args.batch_size,
        epochs=args.epochs,
    )

Embeddings and/or filtered CSV not found, calculating...
Calculating embeddings... for dataset: ecoli_proteins and modality: DNA
Filtering rule for CSV: RNA length <= 1000
Created filtered CSV (RNA-length based).
Original rows : 6348
Kept rows     : 4450 (RNA length <= 1000)
Saved to      : data/datasets/ecoli_proteins_multimodal_filtered_maxlen1000.csv


/Users/sahandhassani/opt/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Embedding ecoli_proteins from DNA:   0%|       | 1/4450 [00:00<24:04,  3.08it/s]

First embedding shape: torch.Size([133, 512])


Embedding ecoli_proteins from DNA:  58%|██▎ | 2589/4450 [10:23<05:33,  5.58it/s]

In [ ]:
import os
import argparse
import torch

from data.dataloaders import get_multimodal_loaders
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.multimodel import build_model
from trainer import RegressionTrainer


def _has_any_embeddings(emb_dir) -> bool:
    if not os.path.isdir(emb_dir):
        return False
    try:
        for fn in os.listdir(emb_dir):
            if fn.endswith(".pt"):
                return True
    except FileNotFoundError:
        return False
    return False

def main(name: str, dataset: str, max_len: int, batch_size: int, epochs: int):
    config = load_config(f"{name}.yml")
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"

    dna_dir = f"embeddings/{config['Dataset']}/DNA/maxlen{max_len}"
    rna_dir = f"embeddings/{config['Dataset']}/RNA/maxlen{max_len}"
    prot_dir = f"embeddings/{config['Dataset']}/Protein/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_dna = not _has_any_embeddings(dna_dir)
    need_rna = not _has_any_embeddings(rna_dir)
    need_prot = not _has_any_embeddings(prot_dir)
    need_embeddings = need_dna or need_rna or need_prot

    if need_filtered_csv or need_embeddings:
        print("Embeddings and/or filtered CSV not found, calculating...")
        for modality in ("DNA", "RNA", "Protein"):
            calculate_embeddings(
                dataset=config["Dataset"],
                modality=modality,
                device=config["device"],
                max_len=max_len,
            )

    print("=" * 60)
    print("Training Configuration (Multimodal):")
    print(f"  Config name: {config.get('name', name)}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Max Len (filter): {max_len}")
    print(f"  Fusion: {config.get('fusion_type', 'concat')}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print(f"  Device: {config['device']}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_multimodal_loaders(
        config["Dataset"],
        batch_size=batch_size,
        max_len=max_len,
    )

    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = ClassificationTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config.get('name', name)}/{config['Dataset']}",
    )

    # entropy reg for MIL (nested config with backwards-compatible fallback)
    lam_entropy = None
    if isinstance(config.get("trainer", None), dict):
        lam_entropy = config["trainer"].get("lam_entropy", None)
    if lam_entropy is None:
        lam_entropy = config.get("lam_entropy", None)

    if lam_entropy is not None:
        trainer.lam_entropy = float(lam_entropy)
        if trainer.lam_entropy > 0:
            print(f"Using MIL entropy regularization: lam_entropy={trainer.lam_entropy}")

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=epochs)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train multimodal model (DNA+RNA+Protein)")
    parser.add_argument(
        "--name",
        type=str,
        default="fusion_mil",
        help="Config file name (without .yml). Options: fusion_concat, fusion_mil, fusion_xattn",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="ecoli_proteins",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1000,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=32,
        help="Batch size.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=500,
        help="Number of training epochs.",
    )
    args, _ = parser.parse_known_args()

    main(
        name=args.name,
        dataset=args.dataset,
        max_len=args.max_len,
        batch_size=args.batch_size,
        epochs=args.epochs,
    )

In [ ]:
import os
import argparse
import torch

from data.dataloaders import get_multimodal_loaders
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.multimodel import build_model
from trainer import RegressionTrainer


def _has_any_embeddings(emb_dir) -> bool:
    if not os.path.isdir(emb_dir):
        return False
    try:
        for fn in os.listdir(emb_dir):
            if fn.endswith(".pt"):
                return True
    except FileNotFoundError:
        return False
    return False

def main(name: str, dataset: str, max_len: int, batch_size: int, epochs: int):
    config = load_config(f"{name}.yml")
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"

    dna_dir = f"embeddings/{config['Dataset']}/DNA/maxlen{max_len}"
    rna_dir = f"embeddings/{config['Dataset']}/RNA/maxlen{max_len}"
    prot_dir = f"embeddings/{config['Dataset']}/Protein/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_dna = not _has_any_embeddings(dna_dir)
    need_rna = not _has_any_embeddings(rna_dir)
    need_prot = not _has_any_embeddings(prot_dir)
    need_embeddings = need_dna or need_rna or need_prot

    if need_filtered_csv or need_embeddings:
        print("Embeddings and/or filtered CSV not found, calculating...")
        for modality in ("DNA", "RNA", "Protein"):
            calculate_embeddings(
                dataset=config["Dataset"],
                modality=modality,
                device=config["device"],
                max_len=max_len,
            )

    print("=" * 60)
    print("Training Configuration (Multimodal):")
    print(f"  Config name: {config.get('name', name)}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Max Len (filter): {max_len}")
    print(f"  Fusion: {config.get('fusion_type', 'concat')}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print(f"  Device: {config['device']}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_multimodal_loaders(
        config["Dataset"],
        batch_size=batch_size,
        max_len=max_len,
    )

    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = ClassificationTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config.get('name', name)}/{config['Dataset']}",
    )

    # entropy reg for MIL (nested config with backwards-compatible fallback)
    lam_entropy = None
    if isinstance(config.get("trainer", None), dict):
        lam_entropy = config["trainer"].get("lam_entropy", None)
    if lam_entropy is None:
        lam_entropy = config.get("lam_entropy", None)

    if lam_entropy is not None:
        trainer.lam_entropy = float(lam_entropy)
        if trainer.lam_entropy > 0:
            print(f"Using MIL entropy regularization: lam_entropy={trainer.lam_entropy}")

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=epochs)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train multimodal model (DNA+RNA+Protein)")
    parser.add_argument(
        "--name",
        type=str,
        default="fusion_xattn",
        help="Config file name (without .yml). Options: fusion_concat, fusion_mil, fusion_xattn",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="ecoli_proteins",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1000,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=32,
        help="Batch size.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=500,
        help="Number of training epochs.",
    )
    args, _ = parser.parse_known_args()

    main(
        name=args.name,
        dataset=args.dataset,
        max_len=args.max_len,
        batch_size=args.batch_size,
        epochs=args.epochs,
    )

# Uni Models:

In [1]:
import torch
import argparse
import os
import pandas as pd
import shutil

from data.dataloaders import get_loaders
from data.subsampler.subsample import subsample_loader
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.unimodel import build_model
from trainer import RegressionTrainer, ClassificationTrainer


def main(name, dataset, max_len):
    config = load_config(f"{name}.yml")
    config["task"] = "classification"
    config["num_classes"] = 3
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"
    emb_dir = f"embeddings/{config['Dataset']}/{config['modality']}/maxlen{max_len}"

    need_embeddings = not os.path.exists(os.path.join(emb_dir, "seq1.pt"))
    need_filtered_csv = not os.path.exists(filtered_csv)

    # Paths
    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"
    emb_dir = f"embeddings/{config['Dataset']}/{config['modality']}/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_embeddings = not os.path.exists(emb_dir)

    # 🔥 If CSV needs to be regenerated, embeddings are INVALID
    if need_filtered_csv and os.path.exists(emb_dir):
        print("Filtered CSV missing → removing stale embeddings")
        shutil.rmtree(emb_dir)

    # 🔥 If embeddings exist but CSV was recreated earlier → force rebuild
    if not need_filtered_csv and os.path.exists(emb_dir):
        df = pd.read_csv(filtered_csv)
        expected_ids = set(df["id"].astype(str))
        existing_ids = {
            f.replace(".pt", "") for f in os.listdir(emb_dir)
            if f.endswith(".pt")
        }

        if not expected_ids.issubset(existing_ids):
            print("Embedding mismatch detected → removing stale embeddings")
            shutil.rmtree(emb_dir)
            need_embeddings = True

    if need_embeddings or need_filtered_csv:
        print("Recomputing embeddings...")
        calculate_embeddings(
            dataset=config["Dataset"],
            modality=config["modality"],
            device=config["device"],
            max_len=max_len,
        )


    print("=" * 60)
    print("Training Configuration:")
    print(f"  Name: {config['name']}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Modality: {config['modality']}")
    print(f"  Max Len (filter): {max_len}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_loaders(
        config["Dataset"],
        32,
        modality=config["modality"],
        max_len=max_len,
    )
    
    print(
        len(train_loader.dataset),
        len(val_loader.dataset),
        len(test_loader.dataset)
    )

    train_loader = subsample_loader(train_loader, fraction=1)
    val_loader   = subsample_loader(val_loader, fraction=1)
    test_loader  = subsample_loader(test_loader, fraction=1)
    
    print(
        len(train_loader.dataset),
        len(val_loader.dataset),
        len(test_loader.dataset)
    )
    
    print("\nInitializing model...")
    model = build_model(config)
    
    print(model)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    
    trainer = ClassificationTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config['name']}/{config['Dataset']}",
    )

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=500)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train unimodal model")
    parser.add_argument(
        "--name",
        type=str,
        default="uni_rna",
        help="Name of config file (without .yml extension) to use. Default: uni_rna",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="ecoli_proteins",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1024,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    args, _ = parser.parse_known_args()
    main(args.name, args.dataset, args.max_len)

Training Configuration:
  Name: uni_rna
  Dataset: ecoli_proteins
  Modality: RNA
  Max Len (filter): 1024

Loading data...
3073 742 721
3073 742 721

Initializing model...
UnimodalClassificationModel(
  (net): TextCNNHead(
    (project): Linear(in_features=640, out_features=640, bias=True)
    (convs): ModuleList(
      (0): Conv1d(640, 100, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): Conv1d(640, 100, kernel_size=(4,), stride=(1,), padding=(2,))
      (2): Conv1d(640, 100, kernel_size=(5,), stride=(1,), padding=(2,))
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (fc): Linear(in_features=300, out_features=3, bias=True)
  )
)
Total number of parameters: 1179443
Trainable parameters: 1179443
Non-trainable parameters: 0

Initializing trainer...

Starting training...

Starting classification training for 500 epochs
Device: cpu
Classes: 3


Training:  10%|███▍                             | 10/97 [00:15<02:12,  1.52s/it]


KeyboardInterrupt: 

In [1]:
import torch
import argparse
import os
import pandas as pd
import shutil

from data.dataloaders import get_loaders
from data.subsampler.subsample import subsample_loader
from utils.load_config import load_config
from utils.calculate_embeddings import calculate_embeddings
from models.unimodel import build_model
from trainer import RegressionTrainer, ClassificationTrainer


def main(name, dataset, max_len):
    config = load_config(f"{name}.yml")
    config["task"] = "regression"
    #config["num_classes"] = 3
    config["Dataset"] = dataset
    config["device"] = "cuda" if torch.cuda.is_available() else "cpu"

    max_len = int(max_len)

    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"
    emb_dir = f"embeddings/{config['Dataset']}/{config['modality']}/maxlen{max_len}"

    need_embeddings = not os.path.exists(os.path.join(emb_dir, "seq1.pt"))
    need_filtered_csv = not os.path.exists(filtered_csv)

    # Paths
    filtered_csv = f"data/datasets/{config['Dataset']}_multimodal_filtered_maxlen{max_len}.csv"
    emb_dir = f"embeddings/{config['Dataset']}/{config['modality']}/maxlen{max_len}"

    need_filtered_csv = not os.path.exists(filtered_csv)
    need_embeddings = not os.path.exists(emb_dir)

    # 🔥 If CSV needs to be regenerated, embeddings are INVALID
    if need_filtered_csv and os.path.exists(emb_dir):
        print("Filtered CSV missing → removing stale embeddings")
        shutil.rmtree(emb_dir)

    # 🔥 If embeddings exist but CSV was recreated earlier → force rebuild
    if not need_filtered_csv and os.path.exists(emb_dir):
        df = pd.read_csv(filtered_csv)
        expected_ids = set(df["id"].astype(str))
        existing_ids = {
            f.replace(".pt", "") for f in os.listdir(emb_dir)
            if f.endswith(".pt")
        }

        if not expected_ids.issubset(existing_ids):
            print("Embedding mismatch detected → removing stale embeddings")
            shutil.rmtree(emb_dir)
            need_embeddings = True

    if need_embeddings or need_filtered_csv:
        print("Recomputing embeddings...")
        calculate_embeddings(
            dataset=config["Dataset"],
            modality=config["modality"],
            device=config["device"],
            max_len=max_len,
        )


    print("=" * 60)
    print("Training Configuration:")
    print(f"  Name: {config['name']}")
    print(f"  Dataset: {config['Dataset']}")
    print(f"  Modality: {config['modality']}")
    print(f"  Max Len (filter): {max_len}")
    print("=" * 60)

    print("\nLoading data...")
    train_loader, val_loader, test_loader = get_loaders(
        config["Dataset"],
        32,
        modality=config["modality"],
        max_len=max_len,
    )
    
    print(
        len(train_loader.dataset),
        len(val_loader.dataset),
        len(test_loader.dataset)
    )

    train_loader = subsample_loader(train_loader, fraction=1)
    val_loader   = subsample_loader(val_loader, fraction=1)
    test_loader  = subsample_loader(test_loader, fraction=1)
    
    print(
        len(train_loader.dataset),
        len(val_loader.dataset),
        len(test_loader.dataset)
    )
    
    print("\nInitializing model...")
    model = build_model(config)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    print(f"Total number of parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

    print("\nInitializing trainer...")
    trainer = RegressionTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=config["device"],
        save_dir=f"./plots/{config['name']}/{config['Dataset']}",
    )

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60 + "\n")

    trainer.train(epochs=500)

    print("\n" + "=" * 60)
    print("Training completed!")
    print("=" * 60)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train unimodal model")
    parser.add_argument(
        "--name",
        type=str,
        default="uni_rna",
        help="Name of config file (without .yml extension) to use. Default: uni_rna",
    )
    parser.add_argument(
        "--dataset",
        type=str,
        default="fungal_expression",
        help=(
            "Dataset to use. Default: fungal_expression. Options: "
            "'mrna_stability', 'ecoli_proteins', 'cov_vaccine_degradation', 'fungal_expression'"
        ),
    )
    parser.add_argument(
        "--max-len",
        type=int,
        default=1024,
        help="Filter threshold: keep only sequences with raw length <= max_len before embedding/training.",
    )
    args, _ = parser.parse_known_args()
    main(args.name, args.dataset, args.max_len)

Training Configuration:
  Name: uni_rna
  Dataset: fungal_expression
  Modality: RNA
  Max Len (filter): 1024

Loading data...
2232 566 445
2232 566 445

Initializing model...


TypeError: __init__() got an unexpected keyword argument 'num_classes'